# 셀레니움 + 뷰티풀숩
1) 셀레니움으로 크롬웹브라우저 실행, 웹페이지 이동
2) 현재 html 페이지를 변수에 저장
3) html 페이지 => 뷰티풀숩 객체로 변경
4) 뷰티풀숩의 메서드를 이용하여 정보 추출
5) 데이타프레임으로 변경 및 csv 저장

# 다나와
- https://www.danawa.com/

In [2]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

### 검색어 입력 후 1페이지에서 제품 정보 데이타 추출하기
- 검색어는 입력문(input) 이용
- 평점과 스펙이 없는 경우에는 공백 처리

### 작업과정
- 1) 셀레니움으로 크롬웹브라우저 실행, 다나와 웹페이지 이동
- 2) 다나와 검색창에 입력 </br> 
- 검색어 입력 : input() 함수를 이용해서 검색어를 입력받은 후 다나와의 검색창에 연결
- 3) 페이지소스 저장 및 뷰티풀숩 객체화
- 4) 검색된결과페이지에서 제품 리스트 추출
- 5) 1개의 상품만 출력
- 6) 1페이지 전체 > 2차원리스트 > 데이타 프레임으로 변경 및 csv 저장

In [4]:
# 입력문(input) 연습
word = input('검색어를 입력해주세요...')
print('입력한 검색어는 ', word, '입니다')

검색어를 입력해주세요... ㅇㅇ


입력한 검색어는  ㅇㅇ 입니다


In [139]:
# 셀레니움으로 크롬웹브라우저 실행, 다나와 웹페이지 이동
url = 'https://www.danawa.com/' 
driver = webdriver.Chrome('chromedriver.exe')
time.sleep(1)
driver.get(url)

C:\Users\pmpkc\AppData\Local\Temp\ipykernel_18548\3895229294.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [ ]:
# # 다나와의 검색창 소스파악
# <div class="search__box search_area_content">
# 		<input type="text" class="search__input" name="k1" id="AKCSearch" autocomplete="off" placeholder="검색어를 입력해주세요." title="검색어 입력" value="">
# 		<input id="module" name="module" value="goods" type="hidden">
# 		<input id="act" name="act" value="dispMain" type="hidden">
# 		<button type="submit" class="search__submit"><span class="blind">검색</span></button>
# 	</div>

In [140]:
# 검색어 입력
word = input('검색어를 입력해주세요...')

# 다나와 검색창 요소에 접근
target = driver.find_element_by_name('k1')

# 검색창 초기화
target.clear()

# 입력받은 검색어를 검색창에 연동
target.send_keys(word)
time.sleep(0.5)

# 전송 및 결과 출력
target.submit()

검색어를 입력해주세요... infiniti


C:\Users\pmpkc\AppData\Local\Temp\ipykernel_18548\4046339174.py:5: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  target = driver.find_element_by_name('k1')


In [127]:
# 3) 페이지 소스 저장 및 뷰티풀숩 객체화**
html_str = driver.page_source
soup = BeautifulSoup(html_str)
#soup

In [134]:
# 4) 검색된 결과 페이지에서 제품 리스트 추출
target_list = soup.select('div.prod_main_info')
len(target_list)

41

In [135]:
# 소스 파악
# ul > product_list > li

In [136]:
# 5) 한개의 상품만 출력
# target_list[0]
# 제품명, 링크, 가격, 평점(공백시 if문으로 교체), 등록일, 스펙 
prod_name = target_list[0].select_one('p.prod_name a').text
prod_url = target_list[0].select_one('p.prod_name a')['href']
price = target_list[0].select_one('p.price_sect a').text


    # 평점이 없다면 공백처리

if target_list[0].select_one('div.point_num strong'):
    score = target_list[0].select_one('div.point_num strong').text
else:
    score =''
    
reg_date = target_list[0].select_one('dl.meta_item.mt_date dd').text


    # 스펙이 없다면 공백처리

if target_list[0].select_one('div.spec_list'):
    spec = target_list[0].select_one('div.spec_list').text.strip().replace('\t','').replace('\n','')
else:
    spec =''

In [137]:
# 추출 확인
print(prod_name, prod_url, price, score, reg_date, spec)

레고 스피드 챔피언 닛산 GT-R 니즈모 (76896) http://prod.danawa.com/info/?pcode=10080045&keyword=nissan+gt-r&cate=19212116 42,810원 4.8 2020.01. 레고 블럭 / 2020년형 / 7세이상 / 조각수: 298 / 시리즈: 스피드챔피언 / Nissan GT-R Nismo


In [138]:
# 1페이지 전체 > 2차원리스트 > 데이타 프레임으로 변경 및 csv 저장
result_list = []

for i in target_list:
    prod_name = i.select_one('p.prod_name a').text
    prod_url = i.select_one('p.prod_name a')['href']
    price = i.select_one('p.price_sect a').text

    if i.select_one('div.point_num strong'):
        score = i.select_one('div.point_num strong').text
    else:
        score =''

    if i.select_one('dl.meta_item.mt_date dd'):
        reg_date = i.select_one('dl.meta_item.mt_date dd').text
    else:
        reg_date =''
    
    
    if i.select_one('div.spec_list'):
        spec = i.select_one('div.spec_list').text.strip().replace('\t','').replace('\n','')
    else:
        spec =''
        
    result_list.append([prod_name, prod_url, price, score, reg_date, spec])

AttributeError: 'NoneType' object has no attribute 'text'

In [125]:
df = pd.DataFrame(result_list, columns = ['제품명', '링크', '가격', '평점', '등록일', '스펙'])
df.tail()

,제품명,링크,가격,평점,등록일,스펙
36,마이크로닉스 WARP GX1 M.2 NVMe,http://prod.danawa.com/info/?pcode=13096874&ke...,"276,700원",4.8,2021.01.,내장형SSD / M.2 (2280) / PCIe3.0x4 (32GT/s) / NVM...
37,마이크론 Crucial P1 M.2 NVMe 대원CTS,http://prod.danawa.com/info/?pcode=6538923&key...,"315,000원",4.7,2018.10.,내장형SSD / M.2 (2280) / PCIe3.0x4 (32GT/s) / NVM...
38,ADATA Ultimate SU800,http://prod.danawa.com/info/?pcode=6476438&key...,"80,500원",4.8,2016.08.,내장형SSD / 6.4cm(2.5형) / SATA3 (6Gb/s) / TLC(기타)...
39,Seagate 아이언울프 510 M.2 NVMe,http://prod.danawa.com/info/?pcode=10869891&ke...,"579,000원",5.0,2020.03.,내장형SSD / M.2 (2280) / PCIe3.0x4 (32GT/s) / NVM...
40,Seagate 바라쿠다 510 M.2 NVMe,http://prod.danawa.com/info/?pcode=9996615&key...,"159,000원",4.8,2019.11.,내장형SSD / M.2 (2280) / PCIe3.0x4 (32GT/s) / NVM...


## 페이지 클릭 테스트

In [ ]:
# # 소스파악
# <div class="paging_number_wrap">
# 					<a href="#" onclick="getPage(1); return false;" class="snum now_spage click_log_page" data-page="1">1</a>
# 					<a href="#" onclick="getPage(2); return false;" class="snum  click_log_page" data-page="2">2</a>
# 					<a href="#" onclick="getPage(3); return false;" class="snum  click_log_page" data-page="3">3</a>
# 					<a href="#" onclick="getPage(4); return false;" class="snum  click_log_page" data-page="4">4</a>
# 					<a href="#" onclick="getPage(5); return false;" class="snum  click_log_page" data-page="5">5</a>
# 					<a href="#" onclick="getPage(6); return false;" class="snum  click_log_page" data-page="6">6</a>
# 					<a href="#" onclick="getPage(7); return false;" class="snum  click_log_page" data-page="7">7</a>
# 					<a href="#" onclick="getPage(8); return false;" class="snum  click_log_page" data-page="8">8</a>
# 					<a href="#" onclick="getPage(9); return false;" class="snum  click_log_page" data-page="9">9</a>
# 					<a href="#" onclick="getPage(10); return false;" class="snum  click_log_page" data-page="10">10</a>
# 				</div>
                
# 'div.paging_number_wrap a'

In [145]:
# 페이징 부분 요소 추출
target_page = driver.find_elements_by_css_selector('div.paging_number_wrap a')
len(target_page)

C:\Users\pmpkc\AppData\Local\Temp\ipykernel_18548\2748258663.py:2: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  target_page = driver.find_elements_by_css_selector('div.paging_number_wrap a')


10

In [146]:
# 3번째 페이지 클릭
target_page[2].click()

## 10페이지 검색 데이타 리스트

- 1) 셀레니움으로 크롬웹브라우저 실행, 다나와 웹페이지 이동
- 2) 다나와 검색창에 입력 </br> 
- 검색어 입력 : input() 함수를 이용해서 검색어를 입력받은 후 다나와의 검색창에 연결
- 3) 페이지소스 저장 및 뷰티풀숩 객체화
- 4) 검색된 결과 페이지에서 제품 리스트 추출
- 5) 1페이지 전체 > 2차원리스트 > 
- 6) 10페이지의 하이퍼 링크 요소추출
- 7) 2 ~ 10 번째 페이지 클릭 후, 3~5과정 반복


- 데이타 프레임으로 변경 및 csv 저장

In [ ]:
# 7) 2~10페이지 버튼 클릭해서 리스트에 데이타 추가


In [8]:
# 셀레니움으로 크롬웹브라우저 실행, 다나와 웹페이지 이동
url = 'https://www.danawa.com/' 
driver = webdriver.Chrome('chromedriver.exe')

driver.get(url)

word = input('검색어를 입력')

# 다나와 검색창 요소에 접근
target = driver.find_element_by_name('k1')

# 검색창 초기화
target.clear()

# 입력받은 검색어를 검색창에 연동
target.send_keys(word)

# 전송 및 결과 출력
target.submit()


# 3) 페이지 소스 저장 및 뷰티풀숩 객체화**
html_str = driver.page_source
soup = BeautifulSoup(html_str)
#soup

# 4) 검색된 결과 페이지에서 제품 리스트 추출
target_list = soup.select('div.prod_main_info')

target_page = driver.find_elements_by_css_selector('div.paging_number_wrap a')

result_list = []
page = 1

C:\Users\pmpkc\AppData\Local\Temp\ipykernel_17236\1446966806.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


검색어를 입력 아이스크림


C:\Users\pmpkc\AppData\Local\Temp\ipykernel_17236\1446966806.py:10: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  target = driver.find_element_by_name('k1')
C:\Users\pmpkc\AppData\Local\Temp\ipykernel_17236\1446966806.py:30: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  target_page = driver.find_elements_by_css_selector('div.paging_number_wrap a')


In [9]:
# 5) 1페이지 전체 > 2차원리스트 > 데이타 프레임으로 변경 및 csv 저장
print(1,'번째 페이지 작업중')

for j in range(1, len(target_page)):
    # 페이지 이동
    target_page = driver.find_elements_by_css_selector('div.paging_number_wrap a')
    time.sleep(1)

    target_page[j].click()
    print(j+1,'번째 페이지 작업중')
    time.sleep(1)

    html_str = driver.page_source
    soup = BeautifulSoup(html_str)

    time.sleep(0.5)
    
    target_list = soup.select('div.prod_main_info')
    
    for i in target_list:
        prod_name = i.select_one('p.prod_name a').text
        prod_url = i.select_one('p.prod_name a')['href']
        price = i.select_one('p.price_sect a').text

        
        # 평점이 없다면 공백처리
        # 평점이 있다면 데이터 추출
        
        if i.select_one('div.point_num strong'):
            score = i.select_one('div.point_num strong').text
        else:
            score =''

        if i.select_one('dl.meta_item.mt_date dd'):
            reg_date = i.select_one('dl.meta_item.mt_date dd').text
        else:
            reg_date =''

        # 스펙이 없다면 공백처리
        # 스펙이 있다면 데이터 추출
        
        if i.select_one('div.spec_list'):
            spec = i.select_one('div.spec_list').text.strip().replace('\t','').replace('\n','')
        else:
            spec =''

        result_list.append([prod_name, prod_url, price, score, reg_date, spec])

1 번째 페이지 작업중


C:\Users\pmpkc\AppData\Local\Temp\ipykernel_17236\603941334.py:6: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  target_page = driver.find_elements_by_css_selector('div.paging_number_wrap a')


2 번째 페이지 작업중
3 번째 페이지 작업중
4 번째 페이지 작업중
5 번째 페이지 작업중
6 번째 페이지 작업중
7 번째 페이지 작업중
8 번째 페이지 작업중
9 번째 페이지 작업중
10 번째 페이지 작업중


In [11]:
# 데이타프레임 확인
df = pd.DataFrame(result_list, columns = ['제품명', '링크', '가격', '평점', '등록일', '스펙'])
df.head()

,제품명,링크,가격,평점,등록일,스펙
0,빙그레 참 붕어싸만코 150ml,http://prod.danawa.com/info/?pcode=5116952&key...,"5,000원",4.7,2017.04.,아이스크림 / 샌드
1,hsOb8NAwYU,/ajax/productPot.ajax.php?key=hsOb8NAwYU,705035원,,,hsOb8NAwYU705035
2,롯데제과 아이스크림 골라담기,http://prod.danawa.com/info/?pcode=12063440&ke...,550원,4.5,2020.08.,아이스크림
3,롯데제과 조안나 바닐라 리치 900ml,http://prod.danawa.com/info/?pcode=11114871&ke...,"4,920원",4.5,2020.04.,"아이스크림 / 컵,파우치"
4,해태제과 아이스가이 피치 200ml,http://prod.danawa.com/info/?pcode=12348929&ke...,"1,500원",4.7,2015.11.,"아이스크림 / 컵,파우치"


In [12]:
df.to_csv('output/다나와_아이스크림.csv', index=False)

In [13]:
ls output

 C 드라이브의 볼륨: Windows-SSD
 볼륨 일련 번호: F2DA-E15C

 C:\Users\pmpkc\Desktop\bsu\workspace202205\1_Scrap\output 디렉터리

2022-06-03  오후 12:51    <DIR>          .
2022-06-03  오후 12:50    <DIR>          ..
2022-05-31  오전 10:45             1,432 bestbook.csv
2022-05-30  오후 05:24            12,444 color.csv
2022-05-30  오전 10:41            14,953 google.html
2022-06-02  오후 12:26               791 google_파이썬.csv
2022-06-02  오후 01:27            15,860 google_파이썬2.csv
2022-06-02  오후 03:08            23,759 melon100.csv
2022-06-03  오후 12:51            81,098 다나와_아이스크림.csv
2022-05-31  오후 01:01             1,514 신과함께1.csv
2022-05-31  오후 01:26             4,986 신과함께2.csv
2022-05-31  오후 02:26            11,555 키드.csv
              10개 파일             168,392 바이트
               2개 디렉터리  170,625,781,760 바이트 남음
